In [ ]:
import os
import re
import logging
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import shutil

# ─── 설정 ──────────────────────────────────────
BASE_URL      = 'https://www.chuncheon.go.kr'
LIST_PATH     = '/cityhall/information-disclosure/business-expenses/chuncheon-cityhall/'
DOWNLOAD_API  = urljoin(BASE_URL, '/_cmm/fms/getImage.do')
BASE_DIR      = 'downloads_by_date'
TIMEOUT       = 20
MAX_WORKERS   = 6
HEADERS       = {'User-Agent': 'Mozilla/5.0'}
ALLOWED_EXT   = ('.xlsx', '.hwp', '.pdf', '.docx')
# ───────────────────────────────────────────────

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')
session = requests.Session()
session.headers.update(HEADERS)

# (1) 게시판 페이지에서 게시글 링크 추출
def fetch_post_links(page):
    url = f"{BASE_URL}{LIST_PATH}?pageIndex={page}"
    r = session.get(url, timeout=TIMEOUT)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, 'html.parser')
    posts = [urljoin(BASE_URL, a['href']) for a in soup.select('a.tit[href*="nttId"]')]
    next_btn = soup.select_one('a.pager-next')
    has_next = bool(next_btn and 'href' in next_btn.attrs)
    return posts, has_next

# (2) 게시글 상세 페이지에서 다운로드 파라미터 + 날짜 추출
def extract_file_params(post_url):
    r = session.get(post_url, timeout=TIMEOUT)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, 'html.parser')

    # 게시글 날짜 추출
    date_tag = soup.select_one('span.day')
    try:
        date_str = datetime.strptime(date_tag.text.strip(), '%Y-%m-%d').strftime('%Y%m%d')
    except:
        date_str = 'unknown'

    # 첨부파일 추출
    files = []
    for a in soup.select('a.file-name[href^="javascript:fn_egov_downFile"]'):
        filename = a.text.strip()
        if not filename.lower().endswith(ALLOWED_EXT):
            continue
        m = re.search(r"fn_egov_downFile\('([^']+)','([^']+)','[^']+','[^']+'\)", a['href'])
        if m:
            fileId, fileSn = m.groups()
            files.append((fileId, fileSn, filename, date_str))
    return files

# (3) 파일 다운로드 실행
def download_file(args):
    fileId, fileSn, filename, date_str = args
    dir_path = os.path.join(BASE_DIR, date_str)
    os.makedirs(dir_path, exist_ok=True)
    dest_path = os.path.join(dir_path, filename)
    if os.path.exists(dest_path):
        logging.info(f"▶ 이미 존재: {filename}")
        return filename, True

    params = {'atchFileId': fileId, 'fileSn': fileSn}
    try:
        r = session.get(DOWNLOAD_API, params=params, headers=HEADERS, timeout=TIMEOUT, stream=True)
        r.raise_for_status()
        with open(dest_path, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
        logging.info(f"✅ 다운로드 완료: {filename}")
        return filename, True
    except Exception as e:
        logging.error(f"❌ 다운로드 실패: {filename} — {e}")
        return filename, False

# (4) 전체 크롤링 실행
def crawl_all():
    all_post_urls = []
    page = 1
    while True:
        logging.info(f"[페이지 {page}] 게시글 목록 요청 중...")
        posts, has_next = fetch_post_links(page)
        if not posts:
            break
        all_post_urls.extend(posts)
        if not has_next:
            break
        page += 1
    logging.info(f"총 게시글 수: {len(all_post_urls)}")

    all_files = []
    with ThreadPoolExecutor(MAX_WORKERS) as ex:
        futures = [ex.submit(extract_file_params, url) for url in all_post_urls]
        for fut in as_completed(futures):
            all_files.extend(fut.result())

    logging.info(f"총 다운로드 대상 파일 수: {len(all_files)}")

    with ThreadPoolExecutor(MAX_WORKERS) as ex:
        futures = [ex.submit(download_file, f) for f in all_files]
        results = list(as_completed(futures))

    success = sum(1 for r in results if r)
    logging.info(f"🎉 전체 완료 — 성공: {success}, 실패: {len(all_files) - success}")

if __name__ == '__main__':
    crawl_all()


In [ ]:
# pdf 형태 raw 파일을 Excel로 변환하는 코드

import os
import pdfplumber
import pandas as pd

# 📁 PDF 파일들이 들어있는 폴더 경로
input_folder = r"C:\bit_esg\python\data_anal\downloads_by_date\raw\pdf"
output_folder = r"C:\bit_esg\python\data_anal\downloads_by_date\raw\pdf\pdf_to_ex"

# 📂 출력 폴더가 없다면 생성
os.makedirs(output_folder, exist_ok=True)

# 📑 PDF 파일 목록 가져오기
pdf_files = [f for f in os.listdir(input_folder) if f.endswith(".pdf")]

# 📋 각 PDF 처리
for pdf_file in pdf_files:
    pdf_path = os.path.join(input_folder, pdf_file)
    excel_filename = os.path.splitext(pdf_file)[0] + ".xlsx"
    excel_path = os.path.join(output_folder, excel_filename)

    all_tables = []

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages, 1):
                tables = page.extract_tables()
                for j, table in enumerate(tables, 1):
                    df = pd.DataFrame(table[1:], columns=table[0])  # 첫 줄을 헤더로 사용
                    sheet_name = f"p{i}_t{j}"
                    all_tables.append((sheet_name[:31], df))  # Excel 시트명은 최대 31자
    except Exception as e:
        print(f"❌ {pdf_file} 처리 중 오류 발생: {e}")
        continue

    # 🧾 Excel로 저장
    if all_tables:
        with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
            for sheet_name, df in all_tables:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"✅ {pdf_file} → {excel_filename} 저장 완료")
    else:
        print(f"⚠️ {pdf_file} 에서 추출된 표가 없습니다.")


In [ ]:
import os
import pandas as pd

# 엑셀 파일이 들어있는 폴더 경로
folder_path = r"C:\bit_esg\python\data_anal\downloads_by_date\2.1.unmatch_실패본"

# 엑셀 확장자 필터링 (.xls, .xlsx)
excel_files = [f for f in os.listdir(folder_path) if f.endswith((".xls", ".xlsx"))]

# 리스트를 DataFrame으로 변환
df = pd.DataFrame(excel_files, columns=["파일명"])

# 저장할 파일 경로
save_path = os.path.join(folder_path, "엑셀파일_목록.xlsx")

# 엑셀로 저장
df.to_excel(save_path, index=False)

print(f"✅ 엑셀 파일로 저장 완료: {save_path}")


✅ 엑셀 파일로 저장 완료: C:\bit_esg\python\data_anal\downloads_by_date\2.1.unmatch_실패본\엑셀파일_목록.xlsx


In [ ]:
import os
import re
import shutil
import pandas as pd

# 1. 📁 경로 설정
folder_path = r"C:\bit_esg\python\data_anal\downloads_by_date\allexcel"
rename_path = os.path.join(folder_path, "rename")
unmatch_path = os.path.join(folder_path, "unmatch")

os.makedirs(rename_path, exist_ok=True)
os.makedirs(unmatch_path, exist_ok=True)

file_list = [f for f in os.listdir(folder_path) if f.endswith((".xls", ".xlsx"))]

# ✅ 이미 사용된 표준파일명 저장용
used_filenames = {}

# 2. ✅ 정규화 함수
def normalize_name(filename):
    result = {
        "원본파일명": filename,
        "연도": None,
        "분기": None,
        "부서": None,
        "표준파일명": None,
        "카테고리": "unmatched",
        "파일처리": ""
    }

    # 연도 추출
    if m := re.search(r"['(](\d{2})년", filename):
        result["연도"] = f"20{m.group(1)}"
    elif m := re.search(r"(20\d{2})년", filename):
        result["연도"] = m.group(1)
    elif m := re.search(r"(20\d{2})[.\-]", filename):
        result["연도"] = m.group(1)

    # 분기 추출
    if m := re.search(r"(\d)분기", filename):
        result["분기"] = f"{m.group(1)}분기"
    elif m := re.search(r"(\d{1,2})[~\-](\d{1,2})월", filename):
        start_month = int(m.group(1))
        if 1 <= start_month <= 3:
            result["분기"] = "1분기"
        elif 4 <= start_month <= 6:
            result["분기"] = "2분기"
        elif 7 <= start_month <= 9:
            result["분기"] = "3분기"
        elif 10 <= start_month <= 12:
            result["분기"] = "4분기"

    # 부서명 추출
    if m := re.findall(r"\((.*?)\)", filename):
        for group in reversed(m):
            if not re.search(r"(분기|월|년|\d{4})", group):
                dept = group
                if "," in dept:
                    dept = dept.split(",")[-1]
                result["부서"] = dept.strip()
                break

    # 표준파일명 생성
    if result["연도"] and result["분기"] and result["부서"]:
        base_name = f"{result['연도']}_{result['분기']}_업무추진비_{result['부서'].replace(' ', '')}.xlsx"

        # 중복 방지 번호 붙이기
        if base_name not in used_filenames:
            used_filenames[base_name] = 1
            final_name = base_name
        else:
            used_filenames[base_name] += 1
            final_name = base_name.replace(".xlsx", f"_{used_filenames[base_name]}.xlsx")

        result["표준파일명"] = final_name
        result["카테고리"] = "matched"

    return result

# 3. 전체 처리 및 파일 복사
results = []

for name in file_list:
    original_path = os.path.join(folder_path, name)
    norm = normalize_name(name)
    
    if norm["카테고리"] == "matched":
        dest_path = os.path.join(rename_path, norm["표준파일명"])
        try:
            shutil.copy2(original_path, dest_path)
            norm["파일처리"] = f"✅ 복사됨 → {norm['표준파일명']}"
        except Exception as e:
            norm["파일처리"] = f"❌ 복사 실패: {str(e)}"
    else:
        dest_path = os.path.join(unmatch_path, name)
        try:
            shutil.copy2(original_path, dest_path)
            norm["파일처리"] = "❌ 정규화 실패 → unmatch 이동"
        except Exception as e:
            norm["파일처리"] = f"❌ 이동 실패: {str(e)}"

    results.append(norm)

# 4. 결과 출력
df_result = pd.DataFrame(results)
matched = df_result[df_result["카테고리"] == "matched"]
unmatched = df_result[df_result["카테고리"] == "unmatched"]

print(f"\n✅ 정규화 및 복사 완료: {len(matched)}개")
print(f"❌ 실패 및 unmatch 이동: {len(unmatched)}개")
print("\n📋 정규화 결과 샘플:")
print(df_result.head(10))

# (선택 저장)
# df_result.to_excel("정규화_복사결과_중복방지포함.xlsx", index=False)


In [7]:
import os
import shutil
import re
from collections import defaultdict

# 폴더 경로
source_folder = r"C:\bit_esg\python\data_anal\downloads_by_date\2.1.unmatch"
target_folder = r"C:\bit_esg\python\data_anal\downloads_by_date\2.1.unmatch_정리본"
fail_folder = r"C:\bit_esg\python\data_anal\downloads_by_date\2.1.unmatch_실패본"

# 폴더 생성
os.makedirs(target_folder, exist_ok=True)
os.makedirs(fail_folder, exist_ok=True)

# 중복 카운터
name_counter = defaultdict(int)

# 정규화 함수
def normalize_filename(filename):
    name = os.path.splitext(filename)[0]

    year_match = re.search(r'(20\d{2}|\d{2})[년도]?', name)
    year = '20' + year_match.group(1) if year_match and len(year_match.group(1)) == 2 else (year_match.group(1) if year_match else '2023')

    quarter_match = re.search(r'(\d)분기', name)
    if not quarter_match:
        return None

    dept_match = re.search(r'\(([^()]+)\)', name)
    if not dept_match:
        dept_match = re.search(r'_(.*?)$', name)
    if not dept_match:
        return None

    quarter = quarter_match.group(1)
    department = dept_match.group(1)

    # 기본 파일명
    base_name = f"{year}년_{quarter}분기_업무추진비_{department}"

    # 중복 방지용 접미사
    name_counter[base_name] += 1
    suffix = f"_{name_counter[base_name]}" if name_counter[base_name] > 1 else ""
    final_name = base_name + suffix + ".xlsx"

    return final_name

# 복사 수행
success, fail = 0, 0

for filename in os.listdir(source_folder):
    if not filename.endswith(".xlsx"):
        continue

    src = os.path.join(source_folder, filename)
    new_name = normalize_filename(filename)

    if new_name:
        dst = os.path.join(target_folder, new_name)
        shutil.copy2(src, dst)
        success += 1
    else:
        dst = os.path.join(fail_folder, filename)
        shutil.copy2(src, dst)
        fail += 1

# 결과 출력
print(f"✅ 중복 방지 포함 정규화 성공: {success}개")
print(f"⚠️ 정규화 실패: {fail}개 → 실패본 폴더에 복사됨")


✅ 중복 방지 포함 정규화 성공: 407개
⚠️ 정규화 실패: 185개 → 실패본 폴더에 복사됨


In [4]:
# 실패본들 파일명 바꿔주는 코드


import os
import pandas as pd

# 엑셀 파일 경로
excel_path = r"엑셀파일_목록.xlsx"
df = pd.read_excel(excel_path)

# 실패본 폴더 경로
fail_folder = r"C:\bit_esg\python\data_anal\downloads_by_date\2.1.unmatch_실패본"

# 파일명 정규화 매핑
name_map = dict(zip(df['파일명'], df['정규화_파일명']))

# 로그 저장
rename_log = []

# 중복 파일명 카운터
existing_names = set(os.listdir(fail_folder))

def get_unique_name(base_name, ext, existing_set):
    counter = 1
    new_name = f"{base_name}{ext}"
    while new_name in existing_set:
        new_name = f"{base_name}_{counter}{ext}"
        counter += 1
    existing_set.add(new_name)
    return new_name

# 파일 목록 가져오기
for file in os.listdir(fail_folder):
    matched = False
    for origin, normalized in name_map.items():
        if origin in file:
            ext = os.path.splitext(file)[1]
            unique_name = get_unique_name(normalized, ext, existing_names)
            src = os.path.join(fail_folder, file)
            dst = os.path.join(fail_folder, unique_name)
            try:
                os.rename(src, dst)
                rename_log.append((file, unique_name, "성공"))
            except Exception as e:
                rename_log.append((file, unique_name, f"실패: {str(e)}"))
            matched = True
            break
    if not matched:
        rename_log.append((file, None, "매칭 실패"))

# 결과 저장
result_df = pd.DataFrame(rename_log, columns=["기존 파일명", "변경된 파일명", "상태"])
result_df.to_excel("파일명_변경_결과.xlsx", index=False)
print("✅ 파일명 변경 완료 및 로그 저장 완료!")



✅ 파일명 변경 완료 및 로그 저장 완료!


In [ ]:
import os
import pandas as pd
import re

folder_path = r"C:\bit_esg\python\data_anal\downloads_by_date\cha"
files = [f for f in os.listdir(folder_path) if f.lower().endswith(".xlsx")]

for file in files:
    file_path = os.path.join(folder_path, file)

    # 엑셀 읽기
    try:
        df_raw = pd.read_excel(file_path, sheet_name=0, header=None, dtype=str)
    except Exception as e:
        print(f"❌ {file}: 파일 읽기 실패 - {e}")
        continue

    # "사용자"라는 헤더가 있는 행 탐색
    header_row_idx = None
    user_col_idx = None

    for idx in range(min(10, len(df_raw))):  # 처음 10행까지만 탐색
        row = df_raw.iloc[idx].fillna("").astype(str)
        for col_idx, cell in enumerate(row):
            if "사용자" in cell:
                header_row_idx = idx
                user_col_idx = col_idx
                break
        if header_row_idx is not None:
            break

    if header_row_idx is None:
        print(f"❌ {file}: '사용자' 헤더를 찾을 수 없음")
        continue

    # 데이터가 시작되는 1행 아래에서 사용자명 추출
    data_start = header_row_idx + 1
    user_values = df_raw.iloc[data_start:, user_col_idx].dropna().astype(str)
    
    if user_values.empty or user_values.str.strip().eq("").all():
        print(f"❌ {file}: '사용자' 값이 없음")
        continue

    user = user_values.iloc[0].strip()

    # 새 파일명 구성
    base, ext = os.path.splitext(file)
    match = re.match(r"^(.*업무추진비)_.*", base)
    base_clean = match.group(1) if match else base
    new_name = f"{base_clean}_{user}{ext}"
    new_path = os.path.join(folder_path, new_name)

    # 파일명 변경
    try:
        os.rename(file_path, new_path)
        print(f"✅ {file} → {new_name}")
    except Exception as e:
        print(f"❌ {file}: 이름 변경 실패 - {e}")


In [26]:
import os
import re

# 대상 폴더 경로
folder_path = r"C:\bit_esg\python\data_anal\downloads_by_date\3.normal"

# 폴더 내 모든 파일 순회
for filename in os.listdir(folder_path):
    # 엑셀 파일만 필터링
    if filename.endswith(".xlsx"):
        # 이미 yyyy년_이면 건너뜀
        if re.match(r'\d{4}년_', filename):
            continue
        
        # 'yyyy년' 뒤에 언더스코어가 없는 경우에만 'yyyy년_'으로 수정
        new_filename = re.sub(r'^(\d{4})년(?!_)', r'\1년_', filename)

        # 파일명이 바뀐 경우만 rename
        if new_filename != filename:
            src = os.path.join(folder_path, filename)
            dst = os.path.join(folder_path, new_filename)
            os.rename(src, dst)
            print(f"✅ {filename} → {new_filename}")


✅ 2024년4분기_업무추진비_효자1동.xlsx → 2024년_4분기_업무추진비_효자1동.xlsx
✅ 2024년4분기_업무추진비_효자2동.xlsx → 2024년_4분기_업무추진비_효자2동.xlsx
✅ 2024년4분기_업무추진비_효자3동.xlsx → 2024년_4분기_업무추진비_효자3동.xlsx
✅ 2024년4분기_업무추진비_후평1동.xlsx → 2024년_4분기_업무추진비_후평1동.xlsx
✅ 2024년4분기_업무추진비_후평2동.xlsx → 2024년_4분기_업무추진비_후평2동.xlsx
✅ 2024년4분기_업무추진비_후평3동.xlsx → 2024년_4분기_업무추진비_후평3동.xlsx
✅ 2025년1분기_업무추진비_감사담당관.xlsx → 2025년_1분기_업무추진비_감사담당관.xlsx
✅ 2025년1분기_업무추진비_강남동.xlsx → 2025년_1분기_업무추진비_강남동.xlsx
✅ 2025년1분기_업무추진비_건강관리과.xlsx → 2025년_1분기_업무추진비_건강관리과.xlsx
✅ 2025년1분기_업무추진비_건설과.xlsx → 2025년_1분기_업무추진비_건설과.xlsx
✅ 2025년1분기_업무추진비_건축과.xlsx → 2025년_1분기_업무추진비_건축과.xlsx
✅ 2025년1분기_업무추진비_경영지원과.xlsx → 2025년_1분기_업무추진비_경영지원과.xlsx
✅ 2025년1분기_업무추진비_경제정책과.xlsx → 2025년_1분기_업무추진비_경제정책과.xlsx
✅ 2025년1분기_업무추진비_관광개발과.xlsx → 2025년_1분기_업무추진비_관광개발과.xlsx
✅ 2025년1분기_업무추진비_관광정책과.xlsx → 2025년_1분기_업무추진비_관광정책과.xlsx
✅ 2025년1분기_업무추진비_교동.xlsx → 2025년_1분기_업무추진비_교동.xlsx
✅ 2025년1분기_업무추진비_교육도시과.xlsx → 2025년_1분기_업무추진비_교육도시과.xlsx
✅ 2025년1분기_업무추진비_국제협력관.xlsx → 2025년_1분기_업무추진비_국제협력관.xlsx
✅ 20

In [1]:
"""
📌 연도별·부서별 분기별 업무추진비 파일 존재 여부 점검 스크립트

이 스크립트는 아래 형식의 파일명이 정리된 폴더를 대상으로 실행됩니다:
    예: 2023년_1분기_업무추진비_총무과.xlsx

기능:
1. 파일명에서 연도, 분기, 부서명을 추출
2. 연도별 + 부서별로 1~4분기 파일 존재 여부를 테이블로 정리
3. 결과를 '연도별_부서별_분기존재여부.xlsx'로 저장
"""

import os
import re
import pandas as pd

# 📁 대상 폴더 경로
folder = r"C:\bit_esg\python\data_anal\downloads_by_date\3.normal"

# 🧩 파일명 패턴 추출 정규식
pattern = re.compile(r'(?P<year>\d{4})년_(?P<quarter>[1-4])분기_업무추진비_(?P<dept>.+?)\.xlsx')

# 📊 추출된 데이터 저장용 리스트
records = []

# 🔍 폴더 내 파일명 순회하며 정보 추출
for fname in os.listdir(folder):
    match = pattern.match(fname)
    if match:
        year = match.group("year")
        quarter = f"{match.group('quarter')}분기"  # '1분기', '2분기' 등
        dept = match.group("dept")
        records.append({"연도": year, "부서명": dept, "분기": quarter})

# 🧾 DataFrame 생성
df = pd.DataFrame(records)

# 📈 피벗 테이블: 연도+부서명 별 분기별 파일 존재 여부 (True/False)
pivot = df.pivot_table(index=["연도", "부서명"], columns="분기", aggfunc="size", fill_value=0)
exists = pivot.applymap(lambda x: x > 0)

# 📅 분기 순서 정렬
ordered_cols = ['1분기', '2분기', '3분기', '4분기']
exists = exists.reindex(columns=ordered_cols)

# 💾 결과 저장
save_path = os.path.join(folder, "연도별_부서별_분기존재여부.xlsx")
exists.to_excel(save_path)

print(f"✅ 연도별-부서별 분기존재여부 저장 완료: {save_path}")


✅ 연도별-부서별 분기존재여부 저장 완료: C:\bit_esg\python\data_anal\downloads_by_date\3.normal\연도별_부서별_분기존재여부.xlsx


C:\Users\bitcamp\AppData\Local\Temp\ipykernel_20896\2547246704.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  exists = pivot.applymap(lambda x: x > 0)


In [ ]:
"""
📌 2025년도 부서 기준으로 연도별(2021~2025) 분기 업무추진비 파일 존재 여부 검증

기능 요약:
1. 파일명(예: 2023년_1분기_업무추진비_총무과.xlsx)에서 연도, 분기, 부서명 추출
2. 2025년에 존재하는 부서 목록만 기준으로 사용
3. 각 부서가 2021~2025년 동안 1~4분기 파일을 모두 갖고 있는지 체크
4. 결과를 Excel로 저장
"""

import os
import re
import pandas as pd

# 📁 대상 폴더 경로
folder = r"C:\bit_esg\python\data_anal\downloads_by_date\3.normal"

# 정규표현식 패턴
pattern = re.compile(r'(?P<year>\d{4})년_(?P<quarter>[1-4])분기_업무추진비_(?P<dept>.+?)\.xlsx')

# 추출 결과
records = []

for fname in os.listdir(folder):
    match = pattern.match(fname)
    if match:
        year = match.group("year")
        quarter = f"{match.group('quarter')}분기"
        dept = match.group("dept")
        records.append({"연도": year, "분기": quarter, "부서명": dept})

df = pd.DataFrame(records)

# ✅ 2025년에 존재한 부서명 추출
dept_2025 = df[df["연도"] == "2025"]["부서명"].unique()

# 2021~2025년 전체 기간 대상
years = [str(y) for y in range(2021, 2026)]
quarters = [f"{q}분기" for q in range(1, 5)]

# 필터링: 2025년 기준 부서만
df_target = df[df["부서명"].isin(dept_2025)]

# 존재 여부 피벗
pivot = df_target.pivot_table(index=["부서명", "연도"], columns="분기", aggfunc="size", fill_value=0)
pivot = pivot.applymap(lambda x: True if x > 0 else False)

# 빠진 연도/분기가 있으면 채우기
pivot = pivot.reindex(
    pd.MultiIndex.from_product([dept_2025, years], names=["부서명", "연도"]),
    fill_value=False
).reindex(columns=quarters, fill_value=False)

# 🎯 각 부서별로 모든 연도·분기가 있는지 확인
pivot["완전여부"] = pivot[quarters].all(axis=1)

# 결과 저장
save_path = os.path.join(folder, "2025기준_부서별_연도별분기존재여부.xlsx")
pivot.to_excel(save_path)

print(f"✅ 검증 완료 → {save_path}")


✅ 검증 완료 → C:\bit_esg\python\data_anal\downloads_by_date\3.normal\2025기준_부서별_연도별분기존재여부.xlsx


C:\Users\bitcamp\AppData\Local\Temp\ipykernel_20896\214069273.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot = pivot.applymap(lambda x: True if x > 0 else False)


In [4]:
import os
import re
import pandas as pd
from collections import defaultdict

# 📁 업무추진비 파일들이 저장된 폴더 경로
folder_path = r"C:\bit_esg\python\data_anal\downloads_by_date\3.normal"

# 📌 연도별 부서명을 저장할 딕셔너리 (중복 제거용 set 사용)
year_dept_map = defaultdict(set)

# 🔍 파일명에서 연도와 부서명을 추출할 정규표현식 (부서명에 숫자 포함 허용)
# 예: "2025년_1분기_업무추진비_후평2동.xlsx" → 2025, 후평2동
pattern = re.compile(r"(20\d{2})년_.*?업무추진비_([가-힣0-9]+)")

# 📂 폴더 내 파일 순회
for filename in os.listdir(folder_path):
    if filename.endswith(('.xls', '.xlsx')):
        match = pattern.search(filename)
        if match:
            year = match.group(1)       # 연도 추출 (예: 2025)
            dept = match.group(2)       # 부서명 추출 (예: 후평2동)
            year_dept_map[year].add(dept)

# 📄 결과를 리스트 형태로 변환 (엑셀 저장용)
records = []
for year in sorted(year_dept_map.keys()):
    for dept in sorted(year_dept_map[year]):
        records.append({"연도": year, "부서명": dept})

# 🧾 데이터프레임 생성
df = pd.DataFrame(records)

# 💾 엑셀 파일로 저장
output_path = os.path.join(folder_path, "년도별_부서명_목록.xlsx")
df.to_excel(output_path, index=False)

# ✅ 완료 메시지 출력
print(f"[완료] 연도별 부서명 리스트를 엑셀로 저장했습니다: {output_path}")


[완료] 연도별 부서명 리스트를 엑셀로 저장했습니다: C:\bit_esg\python\data_anal\downloads_by_date\3.normal\년도별_부서명_목록.xlsx


In [5]:
import os
import re
import pandas as pd
from difflib import get_close_matches

# 📁 1. 업무추진비 파일들이 저장된 폴더 경로
folder_path = r"C:\bit_esg\python\data_anal\downloads_by_date\3.normal"

# 📄 2. 춘천시청 공식 조직도 파일 로드
org_df = pd.read_excel(r"C:\bit_esg\python\data_anal\downloads_by_date\춘천시청_조직도_2025.xlsx")  # 경로는 실제 위치로 수정

# 🔍 3. 정규표현식으로 파일명에서 부서명 추출
# 예: 2025년_1분기_업무추진비_회계과.xlsx → 회계과
pattern = re.compile(r"20\d{2}년_\d{1}분기_업무추진비_(.+?)\.xlsx")

# 🗂 4. 폴더 내 파일명 순회하며 부서명 추출
extracted_departments = []
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        match = pattern.search(filename)
        if match:
            dept = match.group(1)
            extracted_departments.append(dept)

# 📦 5. 중복 제거 후 정렬
unique_extracted_departments = sorted(set(extracted_departments))

# 🎯 6. 조직도 기준 부서명 목록 준비
official_departments = org_df["부서명"].dropna().unique().tolist()

# 🤖 7. 유사도 기반 자동 매핑 (difflib 사용)
mapped = []
for dept in unique_extracted_departments:
    match = get_close_matches(dept, official_departments, n=1, cutoff=0.6)
    mapped.append({
        "파일상 부서명": dept,
        "조직도 기준 유사부서명": match[0] if match else "",
        "수정 제안": match[0] if match else "",
        "자동 매핑 여부": "O" if match else "X"
    })

# 📋 8. 결과를 데이터프레임으로 정리
map_df = pd.DataFrame(mapped)

# 💾 9. 엑셀로 저장
output_path = r"C:\bit_esg\python\data_anal\downloads_by_date\파일명_부서_자동매핑표.xlsx"
map_df.to_excel(output_path, index=False)

print(f"[완료] 매핑표가 저장되었습니다: {output_path}")


[완료] 매핑표가 저장되었습니다: C:\bit_esg\python\data_anal\downloads_by_date\파일명_부서_자동매핑표.xlsx


In [6]:
import os
import re
import pandas as pd
from collections import Counter

# 📁 업무추진비 파일들이 저장된 폴더 경로
folder_path = r"C:\bit_esg\python\data_anal\downloads_by_date\3.normal"

# 🔍 파일명에서 부서명을 추출할 정규표현식
# 예: "2025년_1분기_업무추진비_회계과.xlsx" → 부서명 = 회계과
pattern = re.compile(r"20\d{2}년_\d분기_업무추진비_(.+?)\.(xlsx|xls)")

# 📦 부서명 리스트 수집
dept_list = []
for filename in os.listdir(folder_path):
    if filename.endswith((".xlsx", ".xls")):
        match = pattern.search(filename)
        if match:
            dept_list.append(match.group(1))  # 부서명 추출

# 📊 부서명별 등장 횟수 집계 후 내림차순 정렬
dept_counter = Counter(dept_list)
df = pd.DataFrame(dept_counter.items(), columns=["부서명", "등장횟수"])
df = df.sort_values(by="등장횟수", ascending=False)

# 💾 결과 저장
output_file = os.path.join(folder_path, "전체_부서명_등장순위.xlsx")
df.to_excel(output_file, index=False)

print(f"[완료] 전체 부서명 등장순위를 다음 경로에 저장했습니다: {output_file}")


[완료] 전체 부서명 등장순위를 다음 경로에 저장했습니다: C:\bit_esg\python\data_anal\downloads_by_date\3.normal\전체_부서명_등장순위.xlsx


In [8]:
import os
import pandas as pd
import re

# 📁 업무추진비 엑셀 파일이 저장된 폴더
folder_path = r"C:\bit_esg\python\data_anal\downloads_by_date\3.normal"

# 📌 최종 통합 데이터를 저장할 리스트
merged_data = []

# 📄 폴더 내 모든 .xls 또는 .xlsx 파일 목록 불러오기
file_list = [f for f in os.listdir(folder_path) if f.endswith(('.xls', '.xlsx'))]

# 🔁 각 파일 반복
for file_name in file_list:
    # 🎯 파일명에서 사용자, 연도, 분기 정보 추출
    match = re.match(r"(\d{4})년_(\d+분기)_업무추진비_(.+)\.xls[x]?$", file_name)
    if not match:
        continue
    year, quarter, user = match.groups()
    file_path = os.path.join(folder_path, file_name)

    try:
        # 🗂️ 파일 내 모든 시트 순회
        xls = pd.ExcelFile(file_path)
        for sheet_name in xls.sheet_names:
            df = xls.parse(sheet_name, dtype=str)

            # 헤더 재설정 (병합 셀 보정)
            df.columns = df.iloc[0]
            df = df[1:]

            # 🔍 컬럼명 후보군 정의
            location_keys = ['장소', '사용처']
            purpose_keys = ['집행목적', '사용 내역']
            people_keys = ['대상인원수', '대상인원수(명)', '인원']
            cost_keys = ['금액', '금액(원)']

            # ✅ 실제 존재하는 컬럼 찾기
            location_col = next((col for col in location_keys if col in df.columns), None)
            purpose_col = next((col for col in purpose_keys if col in df.columns), None)
            people_col = next((col for col in people_keys if col in df.columns), None)
            cost_col = next((col for col in cost_keys if col in df.columns), None)

            # ❌ 하나라도 없으면 건너뜀
            if not all([location_col, purpose_col, people_col, cost_col]):
                print(f"[건너뜀] {file_name} - 시트 '{sheet_name}'은 필수 컬럼 누락")
                continue

            # ✅ 정상 데이터만 수집
            for _, row in df.iterrows():
                merged_data.append({
                    '사용자': user,
                    '연도': year,
                    '분기': quarter,
                    '장소': row.get(location_col),
                    '집행목적': row.get(purpose_col),
                    '대상인원수': row.get(people_col),
                    '금액': row.get(cost_col)
                })

    except Exception as e:
        print(f"[에러] {file_name} 처리 중 오류 발생: {e}")

# 📊 결과 DataFrame 생성
result_df = pd.DataFrame(merged_data)

# 💾 저장 경로
output_path = r"C:\bit_esg\python\data_anal\merged_output\업무추진비_통합결과.xlsx"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
result_df.to_excel(output_path, index=False)

print(f"[완료] 필수 컬럼이 존재하는 파일만 저장 완료: {output_path}")


[건너뜀] 2020년_1분기_업무추진비_기획예산과.xlsx - 시트 'Sheet3'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_남산면.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_대중교통과.xlsx - 시트 '업무추진비'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_미래농업과.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_산림과.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_수도운영과.xlsx - 시트 'Sheet2'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_시립청소년도서관.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_약사명동.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_전략산업과.xlsx - 시트 'Sheet2'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_효자1동.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_효자3동.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_1분기_업무추진비_후평2동.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_2분기_업무추진비_기획예산과.xlsx - 시트 'Sheet4'은 필수 컬럼 누락
[건너뜀] 2020년_2분기_업무추진비_대중교통과.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_2분기_업무추진비_미래농업과.xlsx - 시트 '부서운영업무추진비'은 필수 컬럼 누락
[건너뜀] 2020년_2분기_업무추진비_복지정책과.xlsx - 시트 'Sheet1'은 필수 컬럼 누락
[건너뜀] 2020년_2분기_업무추진비_사회적경제과.xlsx - 시트 'Sheet2'은 필수 컬럼 누락
[건너뜀] 2020년_2분기_업무추진비_산림과.xlsx - 

In [26]:
import pandas as pd
import re
from collections import Counter

# 📁 파일 경로 (로컬에서 직접 사용 시 경로 확인 필요)
file_path = r"C:\bit_esg\python\data_anal\merged_output\업무추진비_통합결과.xlsx"

# ✅ 엑셀 파일 불러오기
df = pd.read_excel(file_path)

# ✅ 결측치 제거 및 텍스트 처리
purpose_series = df["집행목적"].dropna().astype(str).str.strip()

# ❌ 식사와 무관한 키워드 리스트
exclude_keywords = [
    '다과', '기념품', '조의금', '구입', '간식', '선물', '회의비', '음료',
    '물품', '장례', '경조사', '청소', '답례', '제작', '현수막', '전달',
    '구매', '물품', '공공요금', '화환', '복사용지', '꽃다발','격려품', '커피', '커피 및 차', 
'접대용 다과']

# ✅ 식사 관련 키워드 포함 필터
include_filters = ['식사', '회식', '간담', '오찬', '만찬','초청',]

# 🔍 전체 단어 추출 및 필터링
words = []
for sentence in purpose_series:
    tokens = re.findall(r'[가-힣]{2,}', sentence)
    filtered = [t for t in tokens if not any(stop in t for stop in exclude_keywords)]
    words.extend(filtered)

# 📊 단어 빈도수 계산
counter = Counter(words)
most_common = counter.most_common(200)

# ✅ 식사 관련 단어만 추출
meal_keywords = sorted({
    word for word, count in most_common
    if any(k in word for k in include_filters)
})

# 📄 결과 DataFrame 저장
meal_df = pd.DataFrame(meal_keywords, columns=["식사_목적_키워드"])
output_path = r"C:\bit_esg\python\data_anal\merged_output\식사_목적_키워드_리스트.xlsx"
meal_df.to_excel(output_path, index=False)

print(f"[완료] 식사 관련 키워드 {len(meal_keywords)}개를 추출했습니다.")
print(f"→ 저장 위치: {output_path}")


[완료] 식사 관련 키워드 12개를 추출했습니다.
→ 저장 위치: C:\bit_esg\python\data_anal\merged_output\식사_목적_키워드_리스트.xlsx


In [27]:
import pandas as pd

# 📁 파일 경로 정의
input_path = r"C:\bit_esg\python\data_anal\merged_output\업무추진비_통합결과.xlsx"
keywords_path = r"C:\bit_esg\python\data_anal\merged_output\식사_목적_키워드_리스트.xlsx"
output_path = r"C:\bit_esg\python\data_anal\merged_output\업무추진비_식사목적_필터링결과.xlsx"

# ✅ 파일 로드
df = pd.read_excel(input_path)
keywords_df = pd.read_excel(keywords_path)

# 📌 키워드 리스트로 변환
meal_keywords = keywords_df["식사_목적_키워드"].dropna().astype(str).tolist()

# 🔍 식사 키워드 포함 여부를 판별하는 함수 정의
def contains_meal_keyword(text):
    if not isinstance(text, str):
        return False
    return any(keyword in text for keyword in meal_keywords)

# ✅ 필터링 수행
filtered_df = df[df["집행목적"].apply(contains_meal_keyword)]

# 💾 결과 저장
filtered_df.to_excel(output_path, index=False)

print(f"[완료] 식사 목적 행 {len(filtered_df)}개 추출 완료.")
print(f"→ 저장 위치: {output_path}")


[완료] 식사 목적 행 9633개 추출 완료.
→ 저장 위치: C:\bit_esg\python\data_anal\merged_output\업무추진비_식사목적_필터링결과.xlsx
